# Hybrid Model Architecture POC - Chat with Base model via Instruct model Bridge

## Project Overview

This Colab notebook provides a proof-of-concept for integrating an AI chatbot to bridge user communication with a local cybersecurity base transformer model (fdtn-ai/Foundation-Sec-8B).

In April 2025, Cisco released Foundation-Sec-8B, a specialized AI base-model for cybersecurity scenarios. The system provides expert-level insights on vulnerabilities, exploits, and security best practices. I wanted to chat with it, but BASE models are NOT ideal for direct conversational use. It's primary function is to predict the next most likely word in a sequence. If you ask it a question like "What is the capital of France?", it might complete the prompt with something like "What is the largest city in France?". It doesn't inherently understand the concept of a question-and-answer format.

### Task Summary:
I wanted to see if I could bridge that limitation by placing a chat/instruct model in front of the base model, so that it could help communicate (reformat the questions) with the base model.

### Key Goals:
- **Utilize BASE Model in Chat**: Use Foundation-Sec-8B in chat.
- **Utilize Chat/Instruct Model as 'Interpreter'**: Use chat/instruct model as 'interpreter' (reformat the questions) between user and base model.
- **Multi-Model Architecture**: Combines specialized and general-purpose models
- **Interactive Chat Interface**: User-friendly Gradio-based interface
- **Performance Optimization**: Efficient model loading and response generation

### Project Status:
- **Development Phase**: Initial implementation (April 2025)
- **Testing Phase**: Comprehensive evaluation
- **Current Status**: Done.
- 
### Key Takeaways:
- Not great results! For tests I used **Llama-3.1-8B-Instruct** as interpreter.
- Though not unexpected, it's not just plug-and-play. An instruct model can not intuitively bridge the gap between Base model and user. Though, I do believe with more prompt contextual training and/or fine-tuning, and/or even using a different instruct/chat model, we might have better success.
- Confidence scoring helps users assess the reliability of information
- Performance metrics are essential for evaluating AI system effectiveness

### Date: April 2025
- **Development Phase**: Initial implementation (April 2025)

### Update August 2025
- **New Release**: Cisco released an Instruct version 
https://huggingface.co/fdtn-ai/Foundation-Sec-8B-Instruct
Test still relevant for other base models.
Alternate solution: Learn how to fine-tune a base model myself to convert to instruct version.

I built a basic next.js test for running:
https://hundley-sec-ai.vercel.app/

### Author: Chris Johnson (kutyadog@gmail.com)

## Setup and Installation

First, let's install the required libraries and set up our environment.

In [ ]:
# Install required libraries
!pip install -q torch transformers gradio

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
import os
import time
from typing import Optional, Tuple

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Model options
BASE_MODEL = "fdtn-ai/Foundation-Sec-8B"
INSTRUCT_MODEL = "meta-llama/Llama-3.1-8B-Instruct"

print("Environment setup complete!")

## Model Loading

Let's load the cybersecurity model and tokenizer. We'll provide options for both the base and instruct versions.

In [ ]:
def load_model(model_name):
    """Load the specified cybersecurity model"""
    try:
        print(f"Loading model: {model_name}")
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # Set pad token if not present
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        # Load model with appropriate settings
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None,
            low_cpu_mem_usage=True
        )
        
        print("Model loaded successfully!")
        return tokenizer, model
        
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Load models
print("Loading cybersecurity models...")
base_tokenizer, base_model = load_model(BASE_MODEL)
instruct_tokenizer, instruct_model = load_model(INSTRUCT_MODEL)

# Set current model to instruct model if available, otherwise use base model
current_model = instruct_model if instruct_model else base_model
current_tokenizer = instruct_tokenizer if instruct_tokenizer else base_tokenizer

if current_model is None:
    print("Error: Could not load any model. Please check your internet connection and try again.")
else:
    print(f"Current model: {INSTRUCT_MODEL if instruct_model else BASE_MODEL}")

## Core Chatbot Class

Let's implement an optimized chatbot class for cybersecurity assistance.

In [ ]:
class SecurityChatbot:
    """Optimized cybersecurity chatbot class"""
    
    def __init__(self, model_name: str = INSTRUCT_MODEL):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.device = device
        self.load_time = 0
        self.is_loaded = False
        
    def load_model(self) -> bool:
        """Load the model with error handling"""
        try:
            start_time = time.time()
            print(f"Loading {self.model_name}...")
            
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            
            # Set pad token if not present
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto" if self.device == "cuda" else None,
                low_cpu_mem_usage=True
            )
            
            self.load_time = time.time() - start_time
            self.is_loaded = True
            print(f"Model loaded in {self.load_time:.2f} seconds")
            return True
            
        except Exception as e:
            print(f"Failed to load model: {e}")
            self.is_loaded = False
            return False
    
    def generate_response(
        self, 
        prompt: str, 
        max_new_tokens: int = 150,
        temperature: float = 0.1,
        top_p: float = 0.9
    ) -> dict:
        """Generate response with optimized parameters"""
        if not self.is_loaded:
            return {
                'response': "Model not loaded. Please wait for model initialization.",
                'confidence': 0.0,
                'sources': []
            }
        
        try:
            # Preprocess prompt
            if not prompt.strip():
                return {
                    'response': "Please enter a valid security question.",
                    'confidence': 0.0,
                    'sources': []
                }
            
            # Format prompt for cybersecurity context
            formatted_prompt = self._format_security_prompt(prompt)
            
            # Tokenize with attention mask
            inputs = self.tokenizer(
                formatted_prompt, 
                return_tensors="pt", 
                padding=True, 
                truncation=True,
                max_length=512
            ).to(self.device)
            
            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    temperature=temperature,
                    top_p=top_p,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )
            
            # Decode and clean response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = response.replace(formatted_prompt, "").strip()
            
            # Post-process response
            response = self._post_process_response(response)
            
            # Calculate confidence based on response quality
            confidence = self._calculate_confidence(response, prompt)
            
            return {
                'response': response if response else "I couldn't generate a meaningful response. Please try rephrasing your question.",
                'confidence': confidence,
                'sources': [self.model_name]
            }
            
        except Exception as e:
            return {
                'response': f"Error: {str(e)}",
                'confidence': 0.0,
                'sources': []
            }
    
    def _format_security_prompt(self, query: str) -> str:
        """Format the prompt for cybersecurity context"""
        return f"""As an expert cybersecurity analyst, provide a detailed analysis of the following query:

Query: {query}

Please provide:
1. Technical explanation of the security concept/vulnerability
2. Potential impacts and risks
3. Mitigation strategies or best practices
4. Related information if applicable

Expert Analysis:"""
    
    def _post_process_response(self, response: str) -> str:
        """Clean up the generated response"""
        # Remove extra whitespace
        response = " ".join(response.split())
        
        # Remove incomplete sentences at the end
        sentences = response.split('. ')
        if len(sentences) > 1 and len(sentences[-1]) < 20:
            response = '. '.join(sentences[:-1]) + '.'
        
        return response
    
    def _calculate_confidence(self, response: str, query: str) -> float:
        """Calculate confidence score for the response"""
        if not response or "Error" in response:
            return 0.0
        
        # Simple heuristics for confidence scoring
        confidence = 0.5  # Base confidence
        
        # Increase confidence for longer, more detailed responses
        if len(response) > 100:
            confidence += 0.2
        if len(response) > 200:
            confidence += 0.1
        
        # Check for security-related keywords
        security_keywords = ['vulnerability', 'exploit', 'mitigation', 'protection', 'attack', 'defense', 'risk', 'threat']
        keyword_count = sum(1 for keyword in security_keywords if keyword.lower() in response.lower())
        confidence += min(keyword_count * 0.05, 0.2)
        
        return min(confidence, 1.0)
    
    def get_model_info(self) -> dict:
        """Get information about the loaded model"""
        return {
            "model_name": self.model_name,
            "device": self.device,
            "load_time": self.load_time,
            "loaded": self.is_loaded
        }

# Initialize the optimized chatbot
security_bot = SecurityChatbot()
if security_bot.load_model():
    model_info = security_bot.get_model_info()
    print(f"Model info: {model_info}")
else:
    print("Failed to initialize security chatbot")

## Example Vulnerability Analysis

Let's test the model with some cybersecurity examples to demonstrate its capabilities.

In [ ]:
def generate_response_old(prompt, model, tokenizer, max_new_tokens=100, temperature=0.1):
    """Legacy response generation function"""
    if model is None or tokenizer is None:
        return "Model not loaded."
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=temperature,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Remove the original prompt from the response
        response = response.replace(prompt, "").strip()
        return response
    except Exception as e:
        return f"Error generating response: {str(e)}"

# Example prompt for CWE to CVE matching
cwe_prompt = """CVE-2021-44228 is a remote code execution flaw in Apache Log4j2 via unsafe JNDI lookups ("Log4Shell"). The CWE is CWE-502.

CVE-2017-0144 is a remote code execution vulnerability in Microsoft's SMBv1 server ("EternalBlue") due to a buffer overflow. The CWE is CWE-119.

CVE-2014-0160 is an information-disclosure bug in OpenSSL's heartbeat extension ("Heartbleed") causing out-of-bounds reads. The CWE is CWE-125.

CVE-2017-5638 is a remote code execution issue in Apache Struts 2's Jakarta Multipart parser stemming from improper input validation of the Content-Type header. The CWE is CWE-20.

CVE-2019-0708 is a remote code execution vulnerability in Microsoft's Remote Desktop Services ("BlueKeep") triggered by a use-after-free. The CWE is CWE-416.

CVE-2015-10011 is a vulnerability about OpenDNS OpenResolve improper log output neutralization. The CWE is"""

print("Testing cybersecurity model with CWE to CVE matching...")
response = generate_response_old(cwe_prompt, current_model, current_tokenizer)
print(f"\nResponse: {response}")

## Interactive Chat Interface

Let's create a Gradio interface for our cybersecurity chatbot.

In [ ]:
def predict(message, chat_history):
    """Generate a response to the user's message"""
    print(f"User message: {message}")
    
    # Generate response using the optimized chatbot
    result = security_bot.generate_response(message, max_new_tokens=150, temperature=0.2)
    
    response = result['response']
    confidence = result['confidence']
    
    # Add confidence indicator
    confidence_emoji = "🟢" if confidence > 0.7 else "🟡" if confidence > 0.4 else "🔴"
    response_with_confidence = f"{confidence_emoji} {response}"
    
    print(f"Model response: {response}")
    print(f"Confidence: {confidence:.2f}")
    
    # Add to chat history
    chat_history.append((message, response_with_confidence))
    
    return "", chat_history

def clear_chat():
    """Clear the chat history"""
    return None, []

In [ ]:
# Create the Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # Cybersecurity AI Assistant
    
    This chatbot uses Cisco's Foundation-Sec-8B model to provide expert-level cybersecurity insights.
    Ask questions about vulnerabilities, exploits, security best practices, and more.
    
    **Model**: Foundation-Sec-8B-Instruct
    **Specialization**: Cybersecurity vulnerabilities, exploits, and defense strategies
    
    **Confidence Indicators:**
    - 🟢 High confidence (detailed, accurate response)
    - 🟡 Medium confidence (partial information)
    - 🔴 Low confidence (limited or uncertain response)
    """)
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=500, label="Security Assistant")
            msg = gr.Textbox(label="Your Security Question", placeholder="Ask about vulnerabilities, exploits, security best practices, etc.")
            with gr.Row():
                submit = gr.Button("Submit")
                clear = gr.Button("Clear Chat")
        
        with gr.Column(scale=1):
            gr.Markdown("**Quick Examples:**")
            examples = gr.Examples(
                examples=[
                    "What is CVE eternal blue and how does it work?",
                    "Explain the difference between CWE-119 and CWE-20",
                    "What are the best practices for preventing Log4Shell attacks?",
                    "How does a buffer overflow vulnerability work?",
                    "What is MITRE ATT&CK framework and how is it used?"
                ],
                inputs=msg
            )
    
    msg.submit(predict, [msg, chatbot], [msg, chatbot])
    submit.click(predict, [msg, chatbot], [msg, chatbot])
    clear.click(clear_chat, outputs=[msg, chatbot])

print("Launching Cybersecurity AI Assistant...")
demo.launch(share=True)

## Alternative Implementation with Gradio Interface

Let's also implement a simpler version using Gradio's Interface component for direct text interaction.

In [ ]:
def predict_simple(message):
    """Simple prediction function for direct text interaction"""
    print(f"Processing message: {message}")
    
    if not message.strip():
        return "Please enter a security question."
    
    # Generate response using the optimized chatbot
    result = security_bot.generate_response(message, max_new_tokens=200, temperature=0.1)
    
    response = result['response']
    confidence = result['confidence']
    
    # Add confidence indicator
    confidence_emoji = "🟢" if confidence > 0.7 else "🟡" if confidence > 0.4 else "🔴"
    response_with_confidence = f"{confidence_emoji} {response}"
    
    return response_with_confidence

# Create a simpler interface
iface = gr.Interface(
    fn=predict_simple,
    inputs=["text"],
    outputs=["text"],
    title="Foundation-Sec-8B Security Assistant",
    description="Ask cybersecurity questions and get expert insights from Cisco's Foundation-Sec-8B model.
    
    **Features:**
    - Specialized cybersecurity knowledge
    - Detailed vulnerability analysis
    - Security best practices
    - Confidence scoring for reliability",
    examples=[
        ["What is CVE-2021-44228 (Log4Shell) and how can I protect my systems?"],
        ["Explain the difference between authentication and authorization in cybersecurity"],
        ["What are the OWASP Top 10 vulnerabilities?"],
        ["How does a zero-day vulnerability work?"],
        ["What is the MITRE ATT&CK framework?"]
    ],
    article="<p>This tool uses Cisco's Foundation-Sec-8B model, a specialized cybersecurity model trained on security-related data.</p>"
)

print("Launching simple security interface...")
iface.launch(share=True)

## Model Comparison

Let's compare the responses from both the base and instruct models to see which performs better.

In [ ]:
def compare_models(test_query):
    """Compare responses from base and instruct models"""
    print(f"\nTesting query: '{test_query}'")
    print("=" * 60)
    
    # Test with base model if available
    if base_model and base_tokenizer:
        base_response = generate_response_old(
            test_query, 
            base_model, 
            base_tokenizer,
            max_new_tokens=100,
            temperature=0.1
        )
        print(f"\nBase Model Response:\n{base_response}")
    else:
        print("\nBase model not available.")
    
    # Test with instruct model if available
    if instruct_model and instruct_tokenizer:
        instruct_response = generate_response_old(
            test_query, 
            instruct_model, 
            instruct_tokenizer,
            max_new_tokens=100,
            temperature=0.1
        )
        print(f"\nInstruct Model Response:\n{instruct_response}")
    else:
        print("\nInstruct model not available.")
    
    print("=" * 60)

# Test queries for model comparison
test_queries = [
    "What is a buffer overflow attack?",
    "Explain SQL injection",
    "How does phishing work?"
]

for query in test_queries:
    compare_models(query)

## Testing and Evaluation

Let's test our optimized chatbot with various cybersecurity queries.

In [ ]:
# Test queries for evaluation
test_queries = [
    "What is Log4Shell (CVE-2021-44228) and how does it work?",
    "Explain the difference between symmetric and asymmetric encryption",
    "What are the OWASP Top 10 web application vulnerabilities?",
    "How does a man-in-the-middle attack work?",
    "What is zero-trust security architecture?",
    "Explain the concept of defense-in-depth in cybersecurity",
    "What is the difference between vulnerability and exploit?",
    "How can I protect my organization from ransomware attacks?"
]

print("Testing Security AI Assistant with various queries:\n")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. Query: {query}")
    print("-" * 60)
    
    # Generate response
    result = security_bot.generate_response(query, max_new_tokens=200)
    response = result['response']
    confidence = result['confidence']
    
    print(f"Response: {response}")
    print(f"Confidence: {confidence:.2f}")
    print("=" * 80)

print("\nTesting complete!")

## Performance Metrics

Let's analyze the performance of our cybersecurity assistant.

In [ ]:
def analyze_performance(test_queries):
    """Analyze the performance of the security chatbot"""
    total_queries = len(test_queries)
    successful_responses = 0
    high_confidence_responses = 0
    avg_confidence = 0
    total_response_length = 0
    
    confidence_scores = []
    response_lengths = []
    
    print("Analyzing performance metrics...\n")
    print("=" * 60)
    
    for i, query in enumerate(test_queries, 1):
        print(f"Processing query {i}/{total_queries}...")
        
        # Generate response
        result = security_bot.generate_response(query, max_new_tokens=200)
        
        if result['response'] and "Error" not in result['response']:
            successful_responses += 1
            
            confidence = result['confidence']
            confidence_scores.append(confidence)
            
            if confidence > 0.7:
                high_confidence_responses += 1
            
            response_length = len(result['response'])
            response_lengths.append(response_length)
            total_response_length += response_length
    
    # Calculate metrics
    success_rate = (successful_responses / total_queries) * 100 if total_queries > 0 else 0
    high_confidence_rate = (high_confidence_responses / successful_responses) * 100 if successful_responses > 0 else 0
    avg_confidence = sum(confidence_scores) / len(confidence_scores) if confidence_scores else 0
    avg_response_length = total_response_length / successful_responses if successful_responses > 0 else 0
    
    # Display metrics
    print("\nPerformance Metrics:")
    print("=" * 60)
    print(f"Total Queries: {total_queries}")
    print(f"Successful Responses: {successful_responses}")
    print(f"Success Rate: {success_rate:.1f}%")
    print(f"High Confidence Responses: {high_confidence_responses}")
    print(f"High Confidence Rate: {high_confidence_rate:.1f}%")
    print(f"Average Confidence: {avg_confidence:.2f}")
    print(f"Average Response Length: {avg_response_length:.0f} characters")
    print("=" * 60)
    
    # Return metrics
    return {
        'total_queries': total_queries,
        'successful_responses': successful_responses,
        'success_rate': success_rate,
        'high_confidence_responses': high_confidence_responses,
        'high_confidence_rate': high_confidence_rate,
        'avg_confidence': avg_confidence,
        'avg_response_length': avg_response_length
    }

# Run performance analysis
performance_metrics = analyze_performance(test_queries)

# Display model info
model_info = security_bot.get_model_info()
print("\nModel Information:")
print("=" * 60)
for key, value in model_info.items():
    print(f"{key.replace('_', ' ').title()}: {value}")
print("=" * 60)